# This file used for finetuning GPT-3 model

DataSet: ****ATIS Airline Travel Information System****

[https://www.kaggle.com/datasets/hassanamin/atis-airlinetravelinformationsystem](https://www.kaggle.com/datasets/hassanamin/atis-airlinetravelinformationsystem)

In [ ]:
import pandas as pd
import numpy as np
np.random.seed(0)

In [ ]:
# Please change this path as per your data file locaion
data = pd.read_csv("/content/atis_intents.csv",header=None)
data.head()

In [ ]:
data.columns = ['intent','text']

In [ ]:
data['intent'].unique()

In [ ]:
data['intent'].nunique()

In [ ]:
data['intent'] = data['intent'].str.replace('#','_')
data['intent'].unique()

In [ ]:
data['intent'] = data['intent'].str.replace('atis_','')
data['intent'].unique()

In [ ]:
data['intent'].value_counts()

In [ ]:
labels = ['flight','ground_service','airfare','abbreviation','flight_time']

In [ ]:
data = data[data["intent"].isin(labels)]
data['intent'].value_counts()

In [ ]:
sample_data = data.groupby('intent').apply(lambda x: x.sample(n=40)).reset_index(drop = True)
sample_data.intent.value_counts()

In [ ]:
sample_data.to_csv("sample_data.csv",index=False)

In [ ]:
sample_data = sample_data[['text','intent']]

In [ ]:

sample_data.head()

In [ ]:
sample_data['text'] = sample_data['text'].str.strip()
sample_data['intent'] = sample_data['intent'].str.strip()

In [ ]:
sample_data['text'] = sample_data['text'] + "\n\nIntent:\n\n"
# sample_data['text'] = "Classify text into on the intent: flight, ground_service, airline, aircraft, flight_time. Text: "+sample_data['text'] + "\n\nIntent:\n\n"
sample_data['intent'] = " "+sample_data['intent'] + " END"
sample_data.head()

In [ ]:
print(sample_data['text'][0])

In [ ]:
print(sample_data['intent'][0])

In [ ]:
sample_data.columns = ['prompt','completion']

In [ ]:
sample_data.to_json("intent_sample.jsonl", orient='records', lines=True)

In [ ]:
# sample_data.to_json("intent_.json", orient='records')

In [ ]:
!pip install --upgrade openai

In [ ]:
!openai tools fine_tunes.prepare_data -f intent_sample.jsonl

In [ ]:
import os
# Set here your OPen ai key
os.environ['OPENAI_API_KEY'] = "key"

In [ ]:
!openai api fine_tunes.create -t "intent_sample_prepared_train.jsonl" -v "intent_sample_prepared_valid.jsonl" -m 'davinci'

In [ ]:
!openai api fine_tunes.follow -i ft-L68gCxl9xH1Cf6JZ7HYGevDV

In [ ]:
# prompt = "Do we have london flight on Monday\n\nIntent:\n\n"
# prompt = "what is the ap57 restriction\n\nIntent:\n\n"
prompt = "show me ground transportation in baltimore\n\nIntent:\n\n"

In [ ]:
import openai
openai.api_key ='key'
response = openai.Completion.create(
  model="davinci:ft-personal-xxxx-xx-xx-xx-xx-xx", # This will be your new trained model with datetime format
  prompt=prompt,
  max_tokens=5,
  temperature=0,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  stop=[" END"]
)
print(response['choices'][0]['text'])